### Características

    RowNumber: índice de cadena de datos
    CustomerId: identificador de cliente único
    Surname: apellido
    CreditScore: valor de crédito
    Geography: país de residencia
    Gender: sexo
    Age: edad
    Tenure: período durante el cual ha madurado el depósito a plazo fijo de un cliente (años)
    Balance: saldo de la cuenta
    NumOfProducts: número de productos bancarios utilizados por el cliente
    HasCrCard: el cliente tiene una tarjeta de crédito (1 - sí; 0 - no)
    IsActiveMember: actividad del cliente (1 - sí; 0 - no)
    EstimatedSalary: salario estimado
### Objetivo

    Exited: El cliente se ha ido (1 - sí; 0 - no)

In [1]:
# Importamos las librerias a utilizar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import roc_auc_score

In [2]:
# Importamos la data a analizar
data = pd.read_csv('../source/Churn.csv')
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


In [3]:
# Revisamos el tipo de DF
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [4]:
# Validamos cantidad de valores vacios hay en la categoria Tenure
data['Tenure'].isna().sum()

909

Podemos darnos cuenta, que en la columna Tenure, se cuenta con valores vacios, analizando el caso, se ha tomado la decicision de reemplazarlo con el valor '0', ya que son personas que no cuentan con el producto de plazo fijo, pero si otros, es por ello que son clientes del banco.

In [5]:
# Reemplazando por 0 los valores vacios
data['Tenure'] = data['Tenure'].fillna(0)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


### Codificamos con OHE

In [6]:
# Una vez codificado, procedo a eliminar columnas que podrian sesgar mi modelo, como el indice y los ID.
data_ohe= pd.get_dummies(data, drop_first=True)
data_ohe = data_ohe.drop(columns=['RowNumber', 'CustomerId'])
data_ohe

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Surname_Abbie,...,Surname_Zotova,Surname_Zox,Surname_Zubarev,Surname_Zubareva,Surname_Zuev,Surname_Zuyev,Surname_Zuyeva,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,False,...,False,False,False,False,False,False,False,False,False,False
1,608,41,1.0,83807.86,1,0,1,112542.58,0,False,...,False,False,False,False,False,False,False,False,True,False
2,502,42,8.0,159660.80,3,1,0,113931.57,1,False,...,False,False,False,False,False,False,False,False,False,False
3,699,39,1.0,0.00,2,0,0,93826.63,0,False,...,False,False,False,False,False,False,False,False,False,False
4,850,43,2.0,125510.82,1,1,1,79084.10,0,False,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5.0,0.00,2,1,0,96270.64,0,False,...,False,False,False,False,False,False,False,False,False,True
9996,516,35,10.0,57369.61,1,1,1,101699.77,0,False,...,False,False,False,False,False,False,False,False,False,True
9997,709,36,7.0,0.00,1,0,1,42085.58,1,False,...,False,False,False,False,False,False,False,False,False,False
9998,772,42,3.0,75075.31,2,1,0,92888.52,1,False,...,False,False,False,False,False,False,False,True,False,True


In [7]:
# Procedo a particionar mi caracteristicas y mi objetivo.
feature = data_ohe.drop(columns='Exited')
target = data_ohe['Exited']

In [8]:
# Escalamos mis variables numericas.
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts','EstimatedSalary']
scaler = StandardScaler()
scaler.fit(data_ohe[numeric])
data_ohe[numeric] = scaler.transform(data_ohe[numeric])
data_ohe

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Surname_Abbie,...,Surname_Zotova,Surname_Zox,Surname_Zubarev,Surname_Zubareva,Surname_Zuev,Surname_Zuyev,Surname_Zuyeva,Geography_Germany,Geography_Spain,Gender_Male
0,-0.326221,0.293517,-0.817441,-1.225848,-0.911583,1,1,0.021886,1,False,...,False,False,False,False,False,False,False,False,False,False
1,-0.440036,0.198164,-1.138838,0.117350,-0.911583,0,1,0.216534,0,False,...,False,False,False,False,False,False,False,False,True,False
2,-1.536794,0.293517,1.110941,1.333053,2.527057,1,0,0.240687,1,False,...,False,False,False,False,False,False,False,False,False,False
3,0.501521,0.007457,-1.138838,-1.225848,0.807737,0,0,-0.108918,0,False,...,False,False,False,False,False,False,False,False,False,False
4,2.063884,0.388871,-0.817441,0.785728,-0.911583,1,1,-0.365276,0,False,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.246488,0.007457,0.146750,-1.225848,0.807737,1,0,-0.066419,0,False,...,False,False,False,False,False,False,False,False,False,True
9996,-1.391939,-0.373958,1.753735,-0.306379,-0.911583,1,1,0.027988,0,False,...,False,False,False,False,False,False,False,False,False,True
9997,0.604988,-0.278604,0.789544,-1.225848,-0.911583,0,1,-1.008643,1,False,...,False,False,False,False,False,False,False,False,False,False
9998,1.256835,0.293517,-0.496044,-0.022608,0.807737,1,0,-0.125231,1,False,...,False,False,False,False,False,False,False,True,False,True


In [9]:
# Particionamos en entrenamiento y validación.
feature_train, feature_valid, target_train, target_valid = train_test_split(feature, target, test_size=0.25, random_state=123)
print(feature_train.shape)
print(feature_valid.shape)
print(target_train.shape)
print(target_valid.shape)

(7500, 2942)
(2500, 2942)
(7500,)
(2500,)


In [10]:
# Aplicamos el modelo de regresion logistica, como modelo base.
model_base = LogisticRegression()
model_base.fit(feature_train, target_train)
model_base_predict = model_base.predict(feature_valid)

c:\Users\Usuario\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [11]:
# Aplicamos nuestra matriz de confusion, con el objetivo de ver cuantas veces el modelo clasifico correctamente las clases
mat_base = pd.crosstab(target_valid, model_base_predict, margins=True)
mat_base

col_0,0,1,All
Exited,,,
0,1956,27,1983
1,490,27,517
All,2446,54,2500


In [12]:
# Extraemos las principales metricas
print(metrics.classification_report(target_valid, model_base_predict))

              precision    recall  f1-score   support

           0       0.80      0.99      0.88      1983
           1       0.50      0.05      0.09       517

    accuracy                           0.79      2500
   macro avg       0.65      0.52      0.49      2500
weighted avg       0.74      0.79      0.72      2500



Nuestro modelo base, clasifica muy bien a los clientes que NO se han ido, pero, para los clientes que se han ido NO los clasifica muy bien, esto se debe a que hay un desequilibrio de clases muy notorio. Asimismo, nuestro accuracy es aceptable con un 78% de aciertos del modelo base, tomando en cuenta que esta muy sesgado es por ello que tenemos un buen accuracy.